In [16]:
import os
import time
from openai import AzureOpenAI


client = AzureOpenAI(
    azure_endpoint="https://team2-openai-eastus2.openai.azure.com/", 
    api_key="WVyPxTvMy6Z3uAcZQvBi41lmMfpgv75zkiKO8egAhqFonAf0pjSbJQQJ99ALACHYHv6XJ3w3AAABACOG0SVa", 
    api_version="2024-05-01-preview"  
)


def gpt_response(user_input):
    try:
        
        assistant = client.beta.assistants.create(
            model="team2-gpt",  
            instructions="""
                ## Role
                You are an author who writes dream interpretations and short stories based on the user's input. Respond in the following JSON format.

                ## JSON Format
                {
                "genre": "",
                "tone": "",
                "viewpoint": "",
                "interpretation": "",
                "keywords": [],
                "story": "",
                "summary": ""
                }

                ---

                ## Instructions
                1. The **interpretation** and **story** must always be written in **Korean**.
                2. The interpretation should always use polite and formal language (**존댓말**) and be limited to approximately **2 sentences**.
                3. The **story** must be a complete narrative written in Korean and should be approximately **800 Korean characters long**.
                4. The story must avoid any violent, graphic, or sexual content, including references to blood, death, or harm.
                5. Use RAG (retrieval-augmented generation) to find symbolic meanings of dreams and incorporate them into the story naturally.
                6. If no relevant data is found via RAG, create an original story based on the user's dream input while strictly adhering to all restrictions.
                7. Extract **5–8 key keywords** from the story that can assist in generating an image.
                8. Create a short **summary** of the story in English after writing it. The summary will be used as an image generation prompt, so it must adhere to these conditions:
                - Avoid any violent, graphic, or sexual language or imagery.
                - Use positive, neutral, and descriptive language suitable for image generation.
                - Replace any violent or graphic elements with neutral or abstract descriptions (e.g., 'a tense encounter' instead of 'a stabbing').

                ---

                ## Restrictions
                1. Do not include any references to violence, blood, death, harm, or sexual content in any part of the output.
                2. Avoid mentioning "dream" as a theme or referring to waking up from a dream.
                3. The summary must strictly avoid violent or inappropriate imagery and focus on safe and creative descriptions.

                ---

                ## Anonymity Guidelines
                1. Do not mention real-world locations (cities, countries, specific place names) or famous people (real individuals or well-known characters).
                2. Replace user-provided names or locations with fictional names or abstract descriptions.

                ---

                ## Additional Safety Measures
                - Before generating the `summary`, apply an additional filtering step:
                - Scan for sensitive or inappropriate terms and replace them with neutral alternatives.
                - Ensure that all outputs are free from offensive content before finalizing.

                ---

                ### Important
                Adhere strictly to all instructions and restrictions outlined above to ensure outputs are safe, ethical, and appropriate for use.

                """,
            tools=[{"type":"file_search"}]  # 파일 검색 도구 활성화
        )
        
        # Thread 생성 및 사용자 메시지 추가
        thread = client.beta.threads.create(
            messages=[
                {
                    "role": "user",
                    "content": user_input,
                    # 첨부 파일과 벡터 저장소 ID 연결
                    "attachments": [
                        {
                            "file_id": "assistant-jNHGgihUisB443IusAHq7qHg",  # 업로드된 파일의 ID로 교체 필요
                            "tools": [{"type": "file_search"}]
                        }
                    ]
                }
            ]
        )
        
        
        # Assistant 실행
        run = client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=assistant.id
        )
        
    
        # 실행 상태 확인 및 대기
        while run.status in ['queued', 'in_progress', 'cancelling']:
            time.sleep(1)
            run = client.beta.threads.runs.retrieve(
                thread_id=thread.id,
                run_id=run.id
            )
        
      
        # 실행 완료 후 결과 반환
        if run.status == 'completed':
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            
            for message in reversed(messages.data):
                if message.role == 'assistant':
                    return message.content[0].text.value  
        else:
            print(f"Last Error: {run.last_error}")
            print(f"Reason: {run.last_error}")
            return f"Error: Run failed with status {run.status}"
    except Exception as e:
        return f"Error: {str(e)}"


if __name__ == "__main__":
    user_input = input("프롬프트를 입력하세요: ")
    response = gpt_response(user_input)
    print("\nGPT의 답변:")
    print(response)





Last Error: LastError(code='rate_limit_exceeded', message='Rate limit is exceeded. Try again in 55 seconds.')
Reason: LastError(code='rate_limit_exceeded', message='Rate limit is exceeded. Try again in 55 seconds.')

GPT의 답변:
Error: Run failed with status failed
